In [1]:
import salabim as sim

env = sim.Environment(time_unit='seconds')

customer_to_log = 500

#Shopping baskets and carts:
carts = env.Resource('carts', capacity=45) #45
baskets = env.Resource('baskets', capacity=500)#several hundred
cart_basket_distribution = sim.Pdf((carts, 0.8,baskets, 0.2))
customer_basketcart_distribution_monitor = sim.Monitor("Customer basket and cart monitor")

#Clerks
#Bread and cheese use resources as they are working via counters. Checkout likely has to use queue as the customers need to choose the smallest one
bread_clerks = env.Resource('bread_clerks', capacity=4) #4 employees, 1-6 items takes 2 min
bread_time_distribution = sim.Exponential(2*60)
cheese_and_dairy_clerks = env.Resource('bread_clerks', capacity=3) #3 employees, 1 min avg.
cheese_and_dairy_time_distribution = sim.Exponential(1*60)

#Checkouts
number_of_checkouts = 3
time_per_item_distribution = sim.Exponential(1.1)
payment_time_distribution = sim.Uniform(40, 60)

#Distributions of items per customer
fruit_and_vegetables_distribution = sim.Triangular(4, 22,10)  # min=4, mode=10, max=22
meat_and_fish_distribution = sim.Triangular(0, 9, 4)           # min=0, mode=4, max=9
bread_distribution = sim.Triangular(1, 10, 4)                  # min=1, mode=4, max=10
cheese_and_dairy_distribution = sim.Triangular(1, 11, 3)       # min=1, mode=3, max=11
canned_and_packed_food_distribution = sim.Triangular(6, 35, 17)# min=6, mode=17, max=35
frozen_foods_distribution = sim.Triangular(2, 19, 8)           # min=2, mode=8, max=19
drinks_distribution = sim.Triangular(1, 20, 9)                 # min=1, mode=9, max=20

item_taking_distribution = sim.Uniform(20,30) #Time to take item out of shelf

#Route choice distribution
#Routes
route1 = [
"fruit_and_vegetables",
"meat_and_fish",
"bread",
"cheese_and_dairy",
"canned_and_packed_food",
"frozen_foods",
"drinks",
] #ABCDEF, 80% 
route2 = [
"meat_and_fish",
"bread",
"cheese_and_dairy",
"fruit_and_vegetables",
"canned_and_packed_food",
"frozen_foods",
"drinks",
] #BCDEFG, 20%
route_distribution = sim.Pdf((route1, 0.8, route2, 0.2))

class Customer(sim.Component):
    """
    Customer class for the supermarket. 
    Traverses the store to via its route to fulfill its shopping_list, while carrying either a shopping basket or cart.
    """
    def setup(self):
        self.route = route_distribution.sample()
        self.shopping_list = {
            "fruit_and_vegetables": int(round(fruit_and_vegetables_distribution.sample())),
            "meat_and_fish": int(round(meat_and_fish_distribution.sample())),
            "bread": int(round(bread_distribution.sample())),
            "cheese_and_dairy": int(round(cheese_and_dairy_distribution.sample())),
            "canned_and_packed_food": int(round(canned_and_packed_food_distribution.sample())),
            "frozen_foods": int(round(frozen_foods_distribution.sample())),
            "drinks": int(round(drinks_distribution.sample())),
        }
        self.carrying = None
        self.actions_log = []

    def log_action(self, action):
        """Helper function to log an action with the current time."""
        self.actions_log.append((env.now(), action))
        
    def process(self):
        """"
        Process determines what the customer will do. At the start they will take a cart or basket. Afterwards they will traverse their route and take the items they need according to their shopping list. If they have finished their route (when progress is equal to the length of the shopping list), they will go to the checkout.
        """
        self.start_shopping()
        for next_product in self.route:
            if self.shopping_list[next_product]>0:
                self.get_product(next_product)   
        self.go_to_checkout()
            
    def start_shopping(self):
        """ Get either a shopping cart or basket"""
        want_to_carry = cart_basket_distribution.sample()
        self.log_action(f"Entered cart/basket queue for {want_to_carry}")
        self.request(want_to_carry)
        customer_basketcart_distribution_monitor.tally(want_to_carry)
        self.log_action(f"Got {want_to_carry}")
        self.carrying = want_to_carry
        
    def go_to_checkout(self):
        """Proceed to the emptiest queue in the checkout and wait while items are processed. Returns shopping cart/basket afterwards."""
        #enter emptiest queue
        emptiest_queue = min(checkouts, key=lambda checkout: checkout.requesters().length()) 
        self.log_action(f"Entered checkout queue {emptiest_queue}")
        self.request(emptiest_queue) 
        self.log_action(f"Started checking out")
        item_scan_time = sum(time_per_item_distribution.sample() for _ in range(sum(self.shopping_list.values())))
        self.hold(item_scan_time+payment_time_distribution.sample()) #hold the customer for scanning all items and during payment
        self.log_action(f"Finished checking out")
        #return cart/basket (implicit since process finishes)       
        
        #print log if we want to debug
        if customer_to_log:
            if self.name() == f"customer.{customer_to_log}":
                print(self.carrying.claimers().print_info())
                print(f"Customer's Action Log for customer {self.name()}:")
                for time, action in self.actions_log:
                    print(f"At time {time}, customer: {action}")
        
        
    def get_product(self, product):
        """
        Function to get the required number of {product}.
        Customer holds while the products are taken.
        Special cases for cheese_and_dairy and bread as they require clerks.
        """
        if product == "cheese_and_dairy":
            self.log_action(f"requesting cheese and dairy")
            self.request(cheese_and_dairy_clerks)
            self.log_action(f"Being helped for cheese and dairy")
            self.hold(cheese_and_dairy_time_distribution.sample())
            self.log_action(f"Got cheese and dairy")
            self.release(cheese_and_dairy_clerks)
        elif product == "bread":
            self.log_action(f"requesting bread")
            self.request(bread_clerks)
            self.log_action(f"Being helped for bread")
            self.hold(bread_time_distribution.sample())
            self.log_action(f"Got bread")
            self.release(bread_clerks)
        else:
            amount = self.shopping_list[product]
            self.log_action(f"Getting {product}")
            for _ in range(amount):
                self.hold(time_per_item_distribution.sample())
                self.log_action(f"Got {product}")

#customer generation
customer_distribution = [30, 80, 110, 90, 80, 70, 80, 90, 100, 120, 90, 40] #Expected total = 980
for index, customer_count  in enumerate(customer_distribution):
    env.ComponentGenerator(Customer, iat=env.Exponential(3600/customer_count), at=index*60*60, duration=60*60) #assumes time in seconds

#Checkout creation
checkouts = []
for i in range(number_of_checkouts):
    checkouts.append(env.Resource(f"checkout_clerk{i}", capacity = 1)) #3, 1.1s per item avg. payment 40-60s

#Run for the full day (duration is in seconds) + 1 hour to make sure all customers can leave the store.
run_count = 10
for i in range(run_count):
    env.run(duration=60*60*(len(customer_distribution)+1))
    #get statistics
    sim.reset() 

#Output statistics:
# for checkout in checkouts:
#     checkout.length.print_histograms()
#carts.print_statistics()
# carts.print_info()
bread_clerks.print_statistics()
bread_clerks.print_info()
cheese_and_dairy_clerks.print_statistics()
# cheese_and_dairy_clerks.print_info()
#customer_basketcart_distribution_monitor.print_histogram(values=True)

#Monitoring:
#Inputs:
#carts count 
#Cart basket distribution (e.g. promote taking baskets)
#bread time
#cheese time
#number of checkouts

#outputs
#queue lengths (checkout)
#request time duration (cart, milk, bread)
#total stay duration

# import matplotlib.pyplot as plt
# 
# plt.plot(*bread_clerks.requesters().length.tx(), drawstyle="steps-post")
# plt.show()

import pandas as pd
def get_monitor_data():
    monitors = [bread_clerks.requesters().length, cheese_and_dairy_clerks.requesters().length, carts.requesters().length, baskets.claimers().length]
    for i in range(len(checkouts)):
        monitors.append(checkouts[i].requesters().length)
    
    return {i.name():{'mean': i.mean(), 'std': i.std(),'min':i.minimum(), 'max': i.maximum() } for i in monitors}
    
    
get_monitor_data()
    


Queue 0x27e19508d60
  name=claimers of carts
  component(s):
    customer.500         enter_time 22262.595 priority=inf
    customer.504         enter_time 22362.242 priority=inf
    customer.505         enter_time 22407.466 priority=inf
    customer.507         enter_time 22430.997 priority=inf
    customer.510         enter_time 22982.840 priority=inf
    customer.512         enter_time 23093.635 priority=inf
    customer.513         enter_time 23100.925 priority=inf
    customer.515         enter_time 23169.365 priority=inf
    customer.516         enter_time 23182.296 priority=inf
    customer.517         enter_time 23236.414 priority=inf
None
Customer's Action Log for customer customer.500:
At time 22262.595062173812, customer: Entered cart/basket queue for Resource (carts)
At time 22262.595062173812, customer: Got Resource (carts)
At time 22262.595062173812, customer: Getting fruit_and_vegetables
At time 22263.963632728737, customer: Got fruit_and_vegetables
At time 22264.2389613

{'Length of requesters of bread_clerks': {'mean': 0.02483279646477333,
  'std': 0.2614971595901852,
  'min': 0,
  'max': 7},
 'Length of requesters of carts': {'mean': 0.4039292962388933,
  'std': 3.43730260021377,
  'min': 0,
  'max': 42},
 'Length of claimers of baskets': {'mean': 0.6836639638514049,
  'std': 2.568228485866595,
  'min': 0,
  'max': 19},
 'Length of requesters of checkout_clerk0': {'mean': 0.7214892533852736,
  'std': 2.6288772495962007,
  'min': 0,
  'max': 16},
 'Length of requesters of checkout_clerk1': {'mean': 0.689549565031864,
  'std': 2.5487977646469675,
  'min': 0,
  'max': 16},
 'Length of requesters of checkout_clerk2': {'mean': 0.6608704933350857,
  'std': 2.470164097482068,
  'min': 0,
  'max': 16}}

1
